# ПОСТРОЕНИЕ МОДЕЛЕЙ МАШИННОГО ОБУЧЕНИЯ


In [19]:
!pip install deepdish
import deepdish as dd

In [32]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter('ignore')
from sklearn.linear_model import LogisticRegression, SGDClassifier, PassiveAggressiveClassifier, Perceptron, RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, BaggingClassifier, ExtraTreesClassifier, RandomTreesEmbedding
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer as TfidfVec
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from nltk.corpus import stopwords
!pip install optuna

In [35]:
import optuna
from optuna.visualization import *
from scipy.sparse import hstack, csr_matrix, lil_matrix
import matplotlib.pyplot as plt

ЗАДАЧА

In [22]:
def valcounter(ser): # функция оценки распределения по классам
  vc=pd.concat([ser.value_counts(), ser.value_counts(normalize=True).round(3)*100],  axis=1 )
  vc.columns=(['Num.', '%'])
  return vc

Загрузим данные и рассмотрим их. Обратим внимание на порядок распаковки файла deep dish созданного в ноутбуке подготовки данных

In [23]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
dd.io.load('/content/drive/MyDrive/DSets_store')['X_sets'].items()

dict_items([('X_test', <4171x59971 sparse matrix of type '<class 'numpy.float64'>'
	with 314397 stored elements in Compressed Sparse Row format>), ('X_train', <19461x59971 sparse matrix of type '<class 'numpy.float64'>'
	with 1497254 stored elements in Compressed Sparse Row format>), ('X_val', <4170x59971 sparse matrix of type '<class 'numpy.float64'>'
	with 313356 stored elements in Compressed Sparse Row format>)])

In [25]:
dd.io.load('/content/drive/MyDrive/DSets_store')['y_sets'].keys()

dict_keys(['y_test', 'y_train', 'y_val'])

In [26]:
X_test, X_train, X_val =[*dd.io.load('/content/drive/MyDrive/DSets_store')['X_sets'].values()]
y_test, y_train, y_val =[*dd.io.load('/content/drive/MyDrive/DSets_store')['y_sets'].values()]

# Оценка производительности моделей на символьных признаках.


Оценим производительности различных моделей с настройками гиперпараметров по умолчанию


In [27]:
# @title
# списки моделей и их имён для вывода
names = [
    "Logistic Regression",
    "Decision Tree",
    "Linear SVC",
    "Gaussian NB",
    "Random Forest",
    "SGD Classifier",
    "Ridge Classifier",
    "XGBoost",
    "Gradient Boosting Classifier",
    # "Light GBM Classifier",
    "Passive Aggressive Classifier",
    "Perceptron",
    "Bagging Classifier",
    "Extra Trees Classifier",
    "Multinomial NB",
]

models = [
    LogisticRegression(max_iter = 100),
    DecisionTreeClassifier(),
    LinearSVC(),
    GaussianNB(),
    RandomForestClassifier(),
    SGDClassifier(loss = 'hinge'),
    RidgeClassifier(),
    XGBClassifier(),
    GradientBoostingClassifier(),
    # LGBMClassifier(),
    PassiveAggressiveClassifier(),
    Perceptron(),
    BaggingClassifier(),
    ExtraTreesClassifier(),
    MultinomialNB(),
]

In [28]:
# @title
# модуль оценки точности предсказания модели
def mypredictor(mymodel, Xtr, ytr, Xval, yval):
  model=mymodel
  reals=[ytr, yval]
  model.fit(Xtr, ytr)
  preds=model.predict(Xtr), model.predict(Xval)
  preds_acc=[round(accuracy_score(preds[i], reals[i]), 3) for i in range(2)]
  return preds_acc


In [29]:
# @title
scoredict={}

# модуль перебора моделей и фиксации результатов оценки качества модели
for modname, model in enumerate(models):
  try:
    print(names[modname])
    scoredict[names[modname]]=mypredictor(model, X_train,
                      y_train,
                      X_val, y_val)
  except Exception:
    print(f'Метод {model} не может быть реализован')
    continue
    # scoredict[names[modname]]=mypredictor(model, X_train.toarray(),
    #                     y_train,
    #                     X_val.toarray(), y_val)
  #
resdf=pd.DataFrame(scoredict, index=['train', 'val']).T.sort_values(by="val", ascending=False)

display(resdf)

Logistic Regression
Decision Tree
Linear SVC
Gaussian NB
Метод GaussianNB() не может быть реализован
Random Forest
SGD Classifier
Ridge Classifier
XGBoost
Gradient Boosting Classifier
Passive Aggressive Classifier
Perceptron
Bagging Classifier
Extra Trees Classifier
Multinomial NB


,train,val
Ridge Classifier,0.987,0.954
Linear SVC,0.976,0.951
Extra Trees Classifier,1.000,0.933
XGBoost,0.978,0.931
Random Forest,1.000,0.916
Gradient Boosting Classifier,0.919,0.902
Bagging Classifier,0.992,0.899
Decision Tree,1.000,0.870
Passive Aggressive Classifier,0.846,0.839
Multinomial NB,0.762,0.741


Мы видим что при построении модели на признаках матрицы TFI-DF с лучше всего себя показали линейные модели (Классификатор на методе опорных векторов  и гребневая классификация). XGBoost поделил почётное третье место с методом добавочных деревьев

Мы получили хорошие результаты. Посмотрим, можем ли мы увеличить результативность метода применением автоматического подбора параметров. Для этого мы будем использовать  Optuna.

In [ ]:
def objective(trial):
    params = {
                'alpha': trial.suggest_float('reg_alpha', 0.05, 1, step=0.01),
                'solver': trial.suggest_categorical('solver', ['sparse_cg','sag']),
                'random_state': 42,
                }

    model = RidgeClassifier(**params)
    model.fit(X_train, y_train)
    predictions = model.predict(X_val)
    acc = accuracy_score(y_val, predictions)
    return acc

study = optuna.create_study(direction='maximize', study_name="Whole_Ridge")
study.optimize(objective, n_trials=20)

[I 2023-10-18 08:48:12,299] A new study created in memory with name: Whole_Ridge
[I 2023-10-18 08:49:23,410] Trial 0 finished with value: 0.9539568345323741 and parameters: {'reg_alpha': 0.5900000000000001, 'solver': 'sag'}. Best is trial 0 with value: 0.9539568345323741.
[I 2023-10-18 08:50:41,197] Trial 1 finished with value: 0.9539568345323741 and parameters: {'reg_alpha': 0.35, 'solver': 'sag'}. Best is trial 0 with value: 0.9539568345323741.
[I 2023-10-18 08:54:02,510] Trial 2 finished with value: 0.9498800959232614 and parameters: {'reg_alpha': 0.05, 'solver': 'sparse_cg'}. Best is trial 0 with value: 0.9539568345323741.
[I 2023-10-18 08:55:21,027] Trial 3 finished with value: 0.9537170263788969 and parameters: {'reg_alpha': 0.33, 'solver': 'sparse_cg'}. Best is trial 0 with value: 0.9539568345323741.
[I 2023-10-18 08:57:25,021] Trial 4 finished with value: 0.9537170263788969 and parameters: {'reg_alpha': 0.13, 'solver': 'sparse_cg'}. Best is trial 0 with value: 0.953956834532374

Гребневая классификация не показала потенциала к повышению своего качества предсказания, модель дала прогноз максимально возможной для него точности.

Вычислим теперь точности предсказания на валидационной и тестовой выборках.

In [ ]:
print(f'Метрика точности навалидационной выборке {study.best_value}')

In [ ]:
pd.DataFrame(RidgeClassifier(alpha=0.24, solver='sag').get_params(), index=['Params'])

In [ ]:
fullsize_tstres=round(accuracy_score(RidgeClassifier(alpha=0.24, solver='sag').fit(X_train, y_train).predict(X_test), y_test), 4)

In [ ]:
print(f'Метрика точности на валидационной выборке {fullsize_tstres}')

Итак полученная модель предсказывает класс принадлежности с точностью 0.95. Посмотрим как выдерживаются остальные метрики.

In [18]:
pd.DataFrame(classification_report(RidgeClassifier(alpha=0.24, solver='sag').fit(X_train, y_train).predict(X_test), y_test, output_dict=True)).round(2)

,0,1,2,3,accuracy,macro avg,weighted avg
precision,0.97,0.93,0.98,0.93,0.95,0.95,0.96
recall,0.94,0.97,0.98,0.95,0.95,0.96,0.95
f1-score,0.95,0.95,0.98,0.94,0.95,0.96,0.95
support,1639.00,902.00,849.00,781.00,0.95,4171.00,4171.00


In [130]:
plot_optimization_history(study)

Определяем оптимальные значения гиперпараметров

In [123]:
plot_param_importances(study)

In [124]:

plot_parallel_coordinate(study)

In [135]:
plot_slice(study)

Подготавливаем данные и оформляем результаты исследования в форме презентации